In [13]:
%%configure -f
{ "name":"Spark-to-Cosmos_DB_Connector",  
  "conf": {
    "spark.jars.packages": "com.microsoft.azure:azure-cosmosdb-spark_2.4.0_2.11:1.4.0",
    "spark.jars.excludes": "org.scala-lang:scala-reflect"
   }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
6,application_1574361531834_0010,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,application_1574361531834_0009,pyspark,idle,Link,Link,
6,application_1574361531834_0010,pyspark,idle,Link,Link,✔


In [14]:
# Champion Container
readConfig = {
"Endpoint" : "https://lolpredict.documents.azure.com:443/",
"Masterkey" : "lAb4WLi89cpXb7jGeMlLZtXUqvlGJip9NtKxDZMWHKUf0wasuAFknSgTxX3M7RAlK2uyFZjau7AJethdaBRDWw==",
"Database" : "league_extraction",
"Collection" : "final_features_matrix", 
}
final_features_matrix = spark.read.format("com.microsoft.azure.cosmosdb.spark").options(**readConfig).load()

## create feature column

In [15]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler

In [18]:
columns = final_features_matrix.schema.names 
not_col = ['id','match_id','label','_attachments', '_etag', '_rid', '_self', '_ts']
features = [col for col in columns if col not in not_col]
features

['100_CARRY_armorperlevel', '100_CARRY_attackdamageperlevel', '100_CARRY_attackrange', '100_CARRY_attackspeed', '100_CARRY_avg(assists)', '100_CARRY_avg(deaths)', '100_CARRY_avg(gold_earned)', '100_CARRY_avg(kills)', '100_CARRY_avg(totalDamageDealt)', '100_CARRY_avg(totalDamageTaken)', '100_CARRY_avg(totalMinionsKilled)', '100_CARRY_avg(totalTimeCrowdControlDealt)', '100_CARRY_avg(visionScore)', '100_CARRY_hpperlevel', '100_CARRY_points', '100_CARRY_spellblockperlevel', '100_JUNGLE_armorperlevel', '100_JUNGLE_attackdamageperlevel', '100_JUNGLE_attackrange', '100_JUNGLE_attackspeed', '100_JUNGLE_avg(assists)', '100_JUNGLE_avg(deaths)', '100_JUNGLE_avg(gold_earned)', '100_JUNGLE_avg(kills)', '100_JUNGLE_avg(totalDamageDealt)', '100_JUNGLE_avg(totalDamageTaken)', '100_JUNGLE_avg(totalMinionsKilled)', '100_JUNGLE_avg(totalTimeCrowdControlDealt)', '100_JUNGLE_avg(visionScore)', '100_JUNGLE_hpperlevel', '100_JUNGLE_points', '100_JUNGLE_spellblockperlevel', '100_MIDDLE_armorperlevel', '100_MI

In [19]:
vectorAssembler = VectorAssembler(inputCols = features,outputCol='features')

In [20]:
class_df = vectorAssembler.transform(final_features_matrix)
class_df = class_df.select(['features','label'])

## train test split

In [21]:
splits = class_df.randomSplit([0.66, 0.333])
train_df = splits[0]
test_df = splits[1]

In [23]:
train_df.show(10)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[3.0,1.7,525.0,0....|    1|
|[3.0,1.7,525.0,0....|    0|
|[3.0,1.7,525.0,0....|    0|
|[3.0,1.7,525.0,0....|    1|
|[3.0,2.7,550.0,0....|    1|
|[3.0,3.3,525.0,0....|    0|
|[3.0,3.3,525.0,0....|    1|
|[3.3,3.61,550.0,0...|    1|
|[3.4,2.96,600.0,0...|    0|
|[3.4,2.96,600.0,0...|    0|
+--------------------+-----+
only showing top 10 rows

In [24]:
test_df.show(10)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[3.0,2.7,550.0,0....|    0|
|[3.3,3.61,550.0,0...|    1|
|[3.4,2.96,600.0,0...|    1|
|[3.5,2.5,550.0,0....|    0|
|[3.5,3.4,525.0,0....|    0|
|[3.75,3.0,500.0,0...|    1|
+--------------------+-----+

## train

In [25]:
lr = LogisticRegression(maxIter=10)
model = lr.fit(train_df)
# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(model.coefficients))
print("Intercept: " + str(model.intercept))

Coefficients: [-0.998346676767,-0.113957590952,-0.0065639441629,7.3982112494,4.71333644296,-3.72859836592,0.00741313947551,2.47461489664,0.000458706022759,0.000535088340885,0.14636150032,-0.0456449075135,-0.0371211089257,-0.0148450622035,-1.3976036912e-06,0.0,-0.133599247938,0.906534705626,-0.00203527201769,3.72970883436,3.21303638439,-5.91330534639,0.00304965729919,0.473270355636,-4.73799557444e-05,-0.00175511295764,0.108146869703,0.0706932957882,-0.111887364955,-0.0549973175207,2.26926615342e-07,0.808913684511,-0.00554107420811,-0.285009029917,0.000253539021744,-5.2299863104,0.201739411293,-4.97617610985,-0.00755708461823,-2.1765236985,-0.000423305403424,-0.00185747499106,-0.15258713925,-0.0488495135144,0.875571602073,0.0187889960015,7.89472930371e-07,-0.133224517593,0.158981376401,-0.110845603634,0.00055874398158,-1.37739462542,2.03764779286,-1.2771622062,0.00659383180618,3.16983405422,4.62364523951e-05,-0.000492711191203,0.0599072444476,-0.0176851238036,-0.160044669274,0.0036953467

## results

In [26]:
train_sum = model.summary
accuracy = train_sum.accuracy
print("Accuracy {}".format(accuracy))
train_sum.roc.show()
print("areaUnderROC: " + str(train_sum.areaUnderROC))

Accuracy 1.0
+------------------+-------------------+
|               FPR|                TPR|
+------------------+-------------------+
|               0.0|                0.0|
|               0.0|0.16666666666666666|
|               0.0| 0.3333333333333333|
|               0.0|                0.5|
|               0.0| 0.6666666666666666|
|               0.0| 0.8333333333333334|
|               0.0|                1.0|
|0.1111111111111111|                1.0|
|0.2222222222222222|                1.0|
|0.3333333333333333|                1.0|
|0.4444444444444444|                1.0|
|0.5555555555555556|                1.0|
|0.6666666666666666|                1.0|
|0.7777777777777778|                1.0|
|0.8888888888888888|                1.0|
|               1.0|                1.0|
|               1.0|                1.0|
+------------------+-------------------+

areaUnderROC: 1.0

In [27]:
train_sum.predictions.show(2)

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[3.0,1.7,525.0,0....|  1.0|[-7.6787463289860...|[4.62340568837398...|       1.0|
|[3.0,1.7,525.0,0....|  0.0|[9.29959467872086...|[0.99990854706922...|       0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 2 rows

## test prediction

In [29]:
predictions = model.transform(test_df)
predictions.select("prediction","label")
# print("Coefficients: " + str(predictions.coefficients))
# print("Intercept: " + str(predictions.intercept))

DataFrame[prediction: double, label: int]

In [30]:
predictions.show(2)

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[3.0,2.7,550.0,0....|    0|[-0.9126480567783...|[0.28645826870560...|       1.0|
|[3.3,3.61,550.0,0...|    1|[-3.0729266498853...|[0.04423792093296...|       1.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 2 rows

In [31]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [33]:
test_evaluator = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction')
print("AUC on test set {}".format(test_evaluator.evaluate(predictions))) #AUC

AUC on test set 0.7777777777777778

#### Save model

In [35]:
https://leaguesparkhdistoragehmm.blob.core.windows.net/league-spark-2019-11-21t17-57-18-738z/ams/

In [42]:
model.save("wasb://league-spark-2019-11-21t17-57-18-738z@leaguesparkhdistoragehmm.blob.core.windows.net/mlmodels")